# Metric (Lasot and vot)

## 1.1 Success and Precsion

In [12]:
import math
import numpy as np
from typing import List, Iterable, Tuple, Any
import itertools
import sys
sys.path.append('/home/lz/anaconda3/envs/nbconda/lib/python3.9/site-packages/vot/')
from attributee import Float, Integer, Boolean, Include
import os
from vot.tracker import Tracker
from vot.dataset import Sequence
from vot.region import Region, RegionType, calculate_overlaps
from vot.experiment import Experiment
from vot.experiment.multirun import UnsupervisedExperiment
from vot.analysis import SequenceAggregator, Analysis, SeparableAnalysis, \
    MissingResultsException, Measure, Sorting, Curve, Plot, SequenceAggregator, \
    Axes, analysis_registry
from vot.utilities.data import Grid
import numpy as np
from shapely.geometry import Polygon

import logging
import scipy
import scipy.io
log_path = os.path.join(os.getcwd(), 'my.log')
logging.basicConfig(filename=log_path, level=logging.DEBUG, filemode='w', format='%(levelname)s:%(asctime)s:%(message)s', datefmt='%Y-%d-%m %H:%M:%S')

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count




def estimateIOU(box_a, box_b):
    if str(box_b[0]) == 'nan':
        return 0
    areaA = box_a[2] * box_a[3]
    areaB = box_b[2] * box_b[3]

    area_sum = areaA + areaB
    max_a_x = box_a[0] + box_a[2]
    max_a_y = box_a[1] + box_a[3]

    max_b_x = box_b[0] + box_b[2]
    max_b_y = box_b[1] + box_b[3]

    inter_x_max = min(max_a_x, max_b_x)
    inter_y_max = min(max_a_y, max_b_y)
    inter_x_min = max(box_a[0], box_b[0])
    inter_y_min = max(box_a[1], box_b[1])
    inter_w = inter_x_max - inter_x_min
    inter_h = inter_y_max - inter_y_min
    if inter_w<=0 or inter_h<=0:
        inter_area = 0 
    else:
        inter_area = inter_w * inter_h

    overlap = inter_area  / (area_sum - inter_area)
    return overlap





class Success(object):
    """Computes and stores the Success"""

    def __init__(self, n=21, max_overlap=1):
        self.max_overlap = max_overlap
        self.Xaxis = np.linspace(0, self.max_overlap, n)
        self.reset()

    def reset(self):
        self.overlaps = []

    def add_overlap(self, val):
        self.overlaps.append(val)

    @property
    def count(self):
        return len(self.overlaps)

    @property
    def value(self):
        # succ = [
        #     np.sum(i >= thres
        #            for i in self.overlaps).astype(float) / self.count
        #     for thres in self.Xaxis
        # ]
        succ = [
            np.sum(np.fromiter((i >= thres
                   for i in self.overlaps), dtype=float)) / self.count
            for thres in self.Xaxis
        ]
        
    
        return np.array(succ)

    @property
    def average(self):
        if len(self.overlaps) == 0:
            return 0
        return np.trapz(self.value, x=self.Xaxis) * 100 / self.max_overlap


class Sequence_t(object):

    def __init__(self, name: str, dataset='/data1/yjy/rgbd_benchmark/alldata/'):
        self._name = name
        self._dataset = dataset
        self._path = self._dataset + self._name
        self._numframe = self.num_frame

    @property
    def num_frame(self):
        try:
            seq_list = os.listdir(os.path.join(self._path, 'color'))
        except:
            print('error')
        return len(seq_list)
    @property
    def name(self) -> str:
        return self._name

    @property
    def identifier(self) -> str:
        return self._name

    @property
    def dataset(self):
        return self._dataset

    @property
    def gt(self):
        gtfile = os.path.join(self._path, 'groundtruth.txt')
        with open(gtfile, 'r') as f:
            value = np.loadtxt(f, delimiter=',')
        
        return value 
    @property
    def invisible(self):
        full_occlusion = os.path.join(self._path, 'full-occlusion.tag')
        if not os.path.exists(full_occlusion):
            value = np.array([0 for i in range(self._numframe)])
        else:
            with open(full_occlusion, 'r') as f:
                value = np.loadtxt(f)
        return value

    @property
    def num_inivisible(self):
        full_occlusion = os.path.join(self._path, 'full-occlusion.tag')
        if not os.path.exists(full_occlusion):
            value = np.array([0 for i in range(self._numframe)])
        else:
            with open(full_occlusion, 'r') as f:
                value = np.loadtxt(f)
        if 1 in value:
            flag = True
            return flag, self._numframe
        else:
            flag = False
            
            return flag, self._numframe 
    # @property
    # def attribute(self):
    #     detattribute_list = ['aspect-change.tag', 'background-clutter.tag','depth-change.tag','fast-motion.tag', 'dark-scene.tag', 'full-occlusion.tag', 'moving-view.tag',
    #                         'deformable.tag', 'out-of-plane.tag', 'out-of-frame.tag', 'partial-occlusion.tag', 'reflective-target.tag', 'size-change.tag', 'similar-objects.tag', 'unassigned.tag']
    #     cdtbattribute_list = ['fast-motion.tag', 'size-change.tag', 'aspect-change.tag', 'partial-occlusion.tag', 'similar-object.tag', 'out-of-plane.tag', 'depth-change.tag', 'reflective-target.tag',
    #                         'deformable.tag', 'dark-scene.tag', 'unassigned.tag', 'full-occlusion.tag', 'out-of-frame.tag']
    #     allattribute_list = list(set(detattribute_list + cdtbattribute_list))
    #     value_list = []
    #     for attribute in allattribute_list:
    #         attpath = os.path.join(self._path, attribute)
    #         try:
    #             with open(attpath, 'r') as f:
    #                 value = np.loadtxt(f)
    #         except:
    #             value = None
    #         value_list.append(value)
    #     dict_attribute = dict.fromkeys(allattribute_list, value_list)
        



class Tracking(object):
    def __init__(self, name: str, path='/data1/yjy/rgbd_benchmark/all_benchmark/results'):
        self._name = name
        self._path = os.path.join(path, self._name)

        self._prre = PrRe()
        self._seqlist = self.re_list()
        self._numseq = len(self._seqlist)
        self._lackseq = [] 
        self._votpath = os.path.join(self._path, 'rgbd-unsupervised')

    @property
    def name(self) -> str:
        return self._name
    @property
    def prre(self):
        return self._prre
    @property
    def _votseqlist(self):
        return self.vot_list()
        
    def lack(self, seq):
        self._lackseq.append(seq)
    def re_list(self):
        seq_list = []
        all_list = os.listdir(self._path)
        for name in all_list:
            if '_001.txt' in name:
                seq = name.split('_001.txt')[0]
                seq_list.append(seq)
        return seq_list
    def vot_list(self):

        all_list = os.listdir(self._votpath)

        return all_list
    def vot_prebox_conf(self, sequence):
        boxtxt = os.path.join(self._votpath, sequence,'{}_001.txt'.format(sequence))
        try:
            with open(boxtxt, 'r') as f:
                pre_value = np.loadtxt(f, delimiter=',', skiprows=1)
        except:
            logging.debug('use \ t in {}'.format(self._name))
            with open(boxtxt, 'r') as f:
                pre_value = np.loadtxt(f, delimiter='\t', skiprows=1)
        conftxt = os.path.join(self._votpath, sequence, '{}_001_confidence.value'.format(sequence))
        if not os.path.exists(conftxt):
            conftxt = os.path.join(self._votpath, sequence, '{}_001_confidence.txt'.format(sequence))
        
        if not os.path.exists(conftxt):
           confidence = [1 for i in range(len(pre_value))]
        else:
            with open(conftxt, 'r') as f:
                confidence = np.loadtxt(f, skiprows=1)
        return pre_value, confidence 

    def prebox_conf(self, sequence):
        boxtxt = os.path.join(self._path, '{}_001.txt'.format(sequence))
        try:
            with open(boxtxt, 'r') as f:
                pre_value = np.loadtxt(f, delimiter=',', skiprows=1)
        except:
            logging.debug('use \ t in {}'.format(self._name))
            with open(boxtxt, 'r') as f:
                pre_value = np.loadtxt(f, delimiter='\t', skiprows=1)
        conftxt = os.path.join(self._path, '{}_001_confidence.value'.format(sequence))
        if not os.path.exists(conftxt):
            conftxt = os.path.join(self._path, '{}_001_confidence.txt'.format(sequence))
        
        if not os.path.exists(conftxt):
           confidence = [1 for i in range(len(pre_value))]
        else:
            with open(conftxt, 'r') as f:
                confidence = np.loadtxt(f, skiprows=1)
        return pre_value, confidence


    









## 1.2 precision and recall

In [13]:
class PrRe(object):
    """Computes and stores the Success"""

    def __init__(self):
        self.reset()
        self.thresholds = np.linspace(1, 0, 100)
    def reset(self):
        self.overlaps = []
        self.confidence = []
        self.visible = []
        self.depthQ = []
        self.attribute_name = []
        self.attribute_value = [[] for i in range(15)]
        self.lt = []
        self.start_frame = 0
        
    def add_overlap(self, val):
        self.overlaps.append(val)

    def add_list_iou(self, overlap:list):
        self.overlaps = np.concatenate((self.overlaps, overlap))
    
    def add_confidence(self, confidence:list):
        self.confidence = np.concatenate((self.confidence, confidence))
    def add_visible(self, visible:list):
        self.visible = np.concatenate((self.visible, visible))

    def add_depthquality(self, depthQ:list):
        self.depthQ = np.concatenate((self.depthQ, depthQ))

    def add_attribute(self, allattribute:dict):
        attributename = []
        for i, key in enumerate(allattribute):
            attributename.append(key) 
            self.attribute_value[i] = np.concatenate((self.attribute_value[i], allattribute.get(key)))
        self.attribute_name = attributename
        
    def add_LT(self, firstindex, length_frame):
        try:
            start_frame = self.start_frame
            first_invisible = start_frame + firstindex[0][0]
            end_frame = start_frame + length_frame - 1
            self.lt.append([start_frame,first_invisible, end_frame])
            self.start_frame += length_frame
        except:
            start_frame = self.start_frame
            end_frame = start_frame + length_frame - 1
            self.lt.append([start_frame,0, end_frame])
            self.start_frame += length_frame


    @property
    def count(self):
        return len(self.overlaps)

    @property
    def value(self):

        # succ = [
        #     np.sum(np.fromiter((i >= thres
        #            for i in self.overlaps), dtype=float)) / self.count
        #     for thres in self.Xaxis
        # ]
        # return np.array(succ)
        
        n_visible = len([vis for vis in self.visible if vis == True])
        precision = len(self.thresholds) * [float(0)]
        recall = len(self.thresholds) * [float(0)]

        for i, threshold in enumerate(self.thresholds):

            subset = self.confidence >= threshold
            
            if np.sum(subset) == 0:
                precision[i] = 1
                recall[i] = 0
            else:
                try:
                    # precision[i] = np.mean(self.overlaps[subset])
                    precision[i] = np.mean(self.overlaps[subset])
                    recall[i] = np.sum(self.overlaps[subset]) / n_visible
                except:
                    print('exception')
                if precision[i] == np.nan or recall[i] == np.nan:
                    print('nan')
        return precision, recall
    @property
    def value_DQ(self):

        '''
            quality level: low < 0.4   medium >= 0.4 <=0.8 high > 0.8
            all_precision: [high medium low]
            all_reacall: [high medium low]
        '''
        try:
            set1 = self.depthQ <=0.8
            set2 = self.depthQ >= 0.4
            lowQ_set = self.depthQ > 0.8
            mediumQ_set = set1 == set2
            highQ_set = self.depthQ < 0.4
        except:
            print('depth quality error') 
        all_set = [highQ_set,mediumQ_set, lowQ_set]
        all_precision =[]
        all_recall = []
        for qualityset in range(len(all_set)):
            confidence = self.confidence[all_set[qualityset]]
            overlaps = self.overlaps[all_set[qualityset]]
            visible = self.visible[all_set[qualityset]]
            n_visible = len([vis for vis in visible if vis == True])
            precision = len(self.thresholds) * [float(0)]
            recall = len(self.thresholds) * [float(0)]

            for i, threshold in enumerate(self.thresholds):

                subset = confidence >= threshold
                
                if np.sum(subset) == 0:
                    precision[i] = 1
                    recall[i] = 0
                else:
                    try:
                        # precision[i] = np.mean(self.overlaps[subset])
                        precision[i] = np.mean(overlaps[subset])
                        recall[i] = np.sum(overlaps[subset]) / n_visible
                    except:
                        print('exception')
                    if precision[i] == np.nan or recall[i] == np.nan:
                        print('nan')
            all_precision.append(precision)
            all_recall.append(recall)
        return all_precision, all_recall

    '''
        attribute 
    '''
    @property
    def value_AT(self):
        '''
            Attribute calculate
        '''
        all_precision = []
        all_recall = []
        all_tag = self.attribute_name
        all_value = self.attribute_value
        for id in range(len(all_tag)):
            index = all_value[id] ==1
            confidence = self.confidence[index]
            overlaps = self.overlaps[index]
            visible = self.visible[index]
            n_visible = len([vis for vis in visible if vis == True])
            precision = len(self.thresholds) * [float(0)]
            recall = len(self.thresholds) * [float(0)]

            for i, threshold in enumerate(self.thresholds):

                subset = confidence >= threshold
                
                if np.sum(subset) == 0:
                    precision[i] = 1
                    recall[i] = 0
                else:
                    try:
                        # precision[i] = np.mean(self.overlaps[subset])
                        precision[i] = np.mean(overlaps[subset])
                        recall[i] = np.sum(overlaps[subset]) / n_visible
                    except:
                        print('exception')
                    if precision[i] == np.nan or recall[i] == np.nan:
                        print('nan')
            #recall[np.isnan(recall)]=0
            all_precision.append(precision)
            all_recall.append(recall)
        
        return all_tag, all_precision, all_recall

    @property
    def value_LT(self):

        '''
            first invisible frame
            performanc: [0: 1st invisible] [1st invisible: end]
        '''
        n_visible = len([vis for vis in self.visible if vis == True])
        precision = len(self.thresholds) * [float(0)]
        recall = len(self.thresholds) * [float(0)]
        lt_Framelist = self.lt
        all_result = [[],[]]
        before_invisible = len(self.overlaps) * [float(0)]
        after_invisible = len(self.overlaps) * [float(0)]
        for index, framelist in enumerate(lt_Framelist):
            start = framelist[0]
            invisible = framelist[1]
            end = framelist[2]
            if invisible == 0:
                continue

            before_invisible[start:invisible] = (invisible-start) * [float(1)]
            after_invisible[invisible:end] = (end-invisible) * [float(1)]



        before_invisible = np.array(before_invisible)
        after_invisible = np.array(after_invisible)
        for id, frame in enumerate([before_invisible, after_invisible]):
            x_frame = frame == 1
            confidence = self.confidence[x_frame]
            overlaps = self.overlaps[x_frame]
            visible = self.visible[x_frame]
            n_visible = len([vis for vis in visible if vis == True])

            precision = len(self.thresholds) * [float(0)]
            recall = len(self.thresholds) * [float(0)]

            for i, threshold in enumerate(self.thresholds):

                subset = confidence >= threshold
                
                if np.sum(subset) == 0:
                    precision[i] = 1
                    recall[i] = 0
                else:
                    try:
                        # precision[i] = np.mean(self.overlaps[subset])
                        precision[i] = np.mean(overlaps[subset])
                        recall[i] = np.sum(overlaps[subset]) / n_visible
                    except:
                        print('exception')
                    if precision[i] == np.nan or recall[i] == np.nan:
                        print('nan')
          
            all_result[id] = [precision, recall]
        return all_result

    @property
    def value_SRE(self):

        # succ = [
        #     np.sum(np.fromiter((i >= thres
        #            for i in self.overlaps), dtype=float)) / self.count
        #     for thres in self.Xaxis
        # ]
        # return np.array(succ)
        
        n_visible = len([vis for vis in self.visible if vis == True])
        precision = len(self.thresholds) * [float(0)]
        recall = len(self.thresholds) * [float(0)]

        for i, threshold in enumerate(self.thresholds):

            subset = self.confidence >= threshold
            
            if np.sum(subset) == 0:
                precision[i] = 1
                recall[i] = 0
            else:
                try:
                    # precision[i] = np.mean(self.overlaps[subset])
                    precision[i] = np.mean(self.overlaps[subset])
                    recall[i] = np.sum(self.overlaps[subset]) / n_visible
                except:
                    print('exception')
                if precision[i] == np.nan or recall[i] == np.nan:
                    print('nan')
        return precision, recall


    @property
    def fscore(self):
        pr, re = self.value

        # pr_score = abs(np.trapz(pr, self.thresholds))
        pr_score = np.max(pr)
        # re_score = abs(np.trapz(re, self.thresholds))
        re_score = np.max(re)
        # pr_score = np.sum(pr)/100
        # re_score = np.sum(re)/100
        fmeasure = [2*pr[i]*re[i]/(pr[i]+re[i]) for i in range(len(pr))]
        fscore = max(fmeasure)
        return pr_score, re_score, fscore

    '''
        depth quality evaluation
        all_result:[high medium low], all_result[high] = [pr,re,f]
    '''
    @property
    def fscore_DQ(self):
        all_pr, all_re = self.value_DQ
        all_result = []
        for index in range(len(all_pr)):
            pr = all_pr[index]
            re = all_re[index]
            pr_score = np.max(pr)
            # re_score = abs(np.trapz(re, self.thresholds))
            re_score = np.max(re)
            # pr_score = np.sum(pr)/100
            # re_score = np.sum(re)/100
            fmeasure = [2*pr[i]*re[i]/(pr[i]+re[i]) for i in range(len(pr))]
            fscore = max(fmeasure)
            all_result.append(np.array([pr_score, re_score, fscore]))
        return all_result
    '''
        attribute
    '''
    @property
    def fscore_AT(self):
        all_tag, all_pr, all_re = self.value_AT
        all_result = dict()
        all_fscore = []
        tag_list = []
        for index in range(len(all_pr)):
            pr = all_pr[index]
            re = all_re[index]
            pr_score = np.max(pr)
            # re_score = abs(np.trapz(re, self.thresholds))
            re_score = np.max(re)
            # pr_score = np.sum(pr)/100
            # re_score = np.sum(re)/100
            fmeasure = [2*pr[i]*re[i]/(pr[i]+re[i]) for i in range(len(pr))]
            fscore = max(fmeasure)
            all_fscore.append(np.array([fscore]))
            tag_list.append(all_tag[index])
        all_result = dict(zip(tag_list,all_fscore))
        return all_result
    '''
        long term evaluation
        first invisible
    '''
    @property
    def fscore_LT(self):
        '''
            before result: result[0]
            after result: result[1]
        '''
        all_result = self.value_LT
        result = []
        for id in range(len(all_result)):
            pr = all_result[id][0]
            re = all_result[id][1]
            pr_score = np.max(pr)
            # re_score = abs(np.trapz(re, self.thresholds))
            re_score = np.max(re)
            # pr_score = np.sum(pr)/100
            # re_score = np.sum(re)/100
            fmeasure = [2*pr[i]*re[i]/(pr[i]+re[i]) for i in range(len(pr))]
            fmeasure = np.nan_to_num(np.array(fmeasure))
            fscore = np.max(fmeasure)
            result.append(fscore)
        return result
    '''
    long term evaluation
    first invisible
    '''
    @property
    def fscore_SRE(self):
        '''
            before result: result[0]
            after result: result[1]
        '''
        all_result = self.value_SRE
        result = []
        for id in range(len(all_result)):
            pr = all_result[id][0]
            re = all_result[id][1]
            pr_score = np.mean(pr)
            # re_score = abs(np.trapz(re, self.thresholds))
            re_score = np.mean(re)
            # pr_score = np.sum(pr)/100
            # re_score = np.sum(re)/100
            fmeasure = [2*pr[i]*re[i]/(pr[i]+re[i]) for i in range(len(pr))]
            fmeasure = np.nan_to_num(np.array(fmeasure))
            fscore = np.max(fmeasure)
            result.append([pr_score, re_score, fscore])
        return result  
class Recall(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count




## SRE evaluation

In [10]:
def compute_SRE_curves(trajectory: Tracking, sequence: Sequence_t, all_prre: PrRe, tre=1):
    
    #overlaps = np.array(calculate_overlaps(trajectory, sequence.groundtruth(), (sequence.size) if bounded else None))
    prebbox, confidence = trajectory.vot_prebox_conf(sequence.name)
    gt = sequence.gt 

    # firstframe in each sequence
    try:
        overlaps = np.concatenate(([1], np.array([estimateIOU(prebbox[i], gt[i+tre] ) for i in range(len(prebbox))])))
        overlaps[np.isnan(overlaps)]=0
        confidence = np.concatenate(([1], np.array(confidence)))
    except:
        print('ok')


    #n_visible = len([region for region in sequence.groundtruth() if region.type is not RegionType.SPECIAL])
    # sequence.invisible (full-occlusion tag) if invisible= 1 full-occlusion invisible
    visible = np.array(sequence.invisible) < 1
    visible = visible + 0
    '''
        temporal evaluation
    '''

    visible = visible[tre-1:]
    '''
        temporal evaluation
    '''
    try:
        assert len(overlaps) == len(visible) == len(confidence)
    except:
        print("assert not equal")
        print("Error Tracker: {} sequence {} sequence frame {} result frame {}".format(trajectory.name, sequence.name, len(sequence.gt), len(prebbox)))
        return False    

    all_prre.add_list_iou(overlaps)
    all_prre.add_visible(visible)
    all_prre.add_confidence(confidence) 


seq_list = os.listdir('/data1/yjy/rgbd_benchmark/alldata')
seq_list.remove('list.txt')

SRE_workspace = '/data1/yjy/rgbd_benchmark/all_benchmark/SRE_workspace'
sre_list = os.listdir(SRE_workspace)
#sre_tracker = ['DAL', 'CSRDCF', 'TSDM', 'DeT', 'iiau_rgbd', 'sttc_rgbd']
sre_tracker = [ 'CSRDCF','DAL', 'DeT', 'iiau_rgbd', 'sttc_rgbd']
sre_average_fscore = [[] for i in range(len(sre_tracker))]
sre_average_pr = [[] for i in range(len(sre_tracker))]
sre_average_re = [[] for i in range(len(sre_tracker))]
tre_average_fscore = [[] for i in range(len(sre_tracker))]
tre_average_re = [[] for i in range(len(sre_tracker))]
tre_average_pr = [[] for i in range(len(sre_tracker))]
for sre in sre_list:
    if sre=='tre10' or sre =='tre20' or sre=='tre40' or sre=='tre50':
       # all_trackers = [Tracking(tracker) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/results/')]
        # all_trackers = [Tracking(tracker, path=os.path.join(SRE_workspace, sre, 'results')) for i,tracker in enumerate(['DAL'])]
        all_trackers = [Tracking(tracker, path=os.path.join(SRE_workspace, sre, 'results')) for i,tracker in enumerate(sre_tracker)]
        # all_trackers = [Tracking(tracker) for tracker in ['CADMS']]
        all_sequence = [Sequence_t(seq) for seq in seq_list]

        for id, trackers in enumerate(all_trackers):
            print(trackers.name)
            
            for sequence in all_sequence:
          
                if not os.path.exists(os.path.join(trackers._votpath, sequence.name, '{}_001.txt'.format(sequence.name))):
                    continue
                if sequence.name in trackers._votseqlist:
                    compute_SRE_curves(trackers, sequence, trackers._prre, tre=int(sre.split('tre')[1]))
                    #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
                else:
                    trackers.lack(sequence.name)
                    continue
                
            pr,re,fscore = trackers._prre.fscore
            if trackers.name == sre_tracker[id]:
                tre_average_fscore[id].append(fscore)
                tre_average_pr[id].append(pr)
                tre_average_re[id].append(re)
            print('Trackers: {} sre: {}  pr: {} re: {} fscore: {}'.format(trackers.name, sre, pr, re, fscore))
    else:
        continue
# all_trackers = [Tracking(tracker) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/results/')]
    # all_trackers = [Tracking(tracker, path=os.path.join(SRE_workspace, sre, 'results')) for i,tracker in enumerate(sre_tracker)]
    # # all_trackers = [Tracking(tracker) for tracker in ['CADMS']]
    # all_sequence = [Sequence_t(seq) for seq in seq_list]

    # for id, trackers in enumerate(all_trackers):
    #     print(trackers.name)
    #     for sequence in all_sequence:
    #         if not os.path.exists(os.path.join(trackers._votpath, sequence.name, '{}_001.txt'.format(sequence.name))):
    #             continue
    #         if sequence.name in trackers._votseqlist:
    #             compute_SRE_curves(trackers, sequence, trackers._prre)
    #             #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
    #         else:
    #             trackers.lack(sequence.name)
    #             continue
            
    #     pr,re,fscore = trackers._prre.fscore
    #     if trackers.name == sre_tracker[id]:
    #         sre_average_fscore[id].append(fscore)
    #         sre_average_pr[id].append(pr)
    #         sre_average_re[id].append(re)
    #     print('Trackers: {} sre: {}  pr: {} re: {} fscore: {}'.format(trackers.name, sre, pr, re, fscore))
        # logging.info('Trackers: {}  Seq_num: {} frame_num: {}  pr: {}  re: {}  fscore: {}'.format(trackers.name, trackers._numseq, trackers._prre.count, pr, re, fscore))

for i, sre in enumerate(sre_average_fscore):
    trackerID = sre_tracker[i]
    srefscore = np.mean(np.array(sre))
    srepr = np.mean(np.array(sre_average_pr[i]))
    srere = np.mean(np.array(sre_average_re[i]))
    print('Trackers: {} sre pr:{} re: {} fscore: {}'.format(trackerID, srepr, srere, srefscore))

    logging.info('Trackers: {} sre pr:{} re: {} fscore: {}'.format(trackerID, srepr, srere, srefscore))
for i, tre in enumerate(tre_average_fscore):
    trackerID = sre_tracker[i]
    trefscore = np.mean(np.array(tre))
    print('Trackers: {} tre fscore: {}'.format(trackerID, trefscore))
    logging.info('Trackers: {} tre fscore: {}'.format(trackerID, trefscore))

print('ok')

CSRDCF
Trackers: CSRDCF sre: tre40  pr: 0.15838444877501642 re: 0.16559650100564283 fscore: 0.16191020211902546
DAL
Trackers: DAL sre: tre40  pr: 0.730509404810487 re: 0.5165897970906259 fscore: 0.5456375946241129
DeT
Trackers: DeT sre: tre40  pr: 0.6247433821421966 re: 0.5645487317887158 fscore: 0.5792235294508373
iiau_rgbd
Trackers: iiau_rgbd sre: tre40  pr: 1.0 re: 0.6661307278552203 fscore: 0.6689595050853347
sttc_rgbd
Trackers: sttc_rgbd sre: tre40  pr: 0.9972393100710898 re: 0.607476003603953 fscore: 0.6031746719142198
CSRDCF


Traceback (most recent call last):
  File "/tmp/ipykernel_42459/766108014.py", line 29, in compute_SRE_curves
    assert len(overlaps) == len(visible) == len(confidence)
AssertionError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1078, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 297, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/home/lz/anaconda3/envs/nbconda/lib/python3.9/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 1976, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/home/lz/anaconda3/envs/nbconda/lib/python3.9/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2011, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 

## Long-term evaluation


In [ ]:
def compute_LT_curves(trajectory: Tracking, sequence: Sequence_t, all_prre: PrRe):
    
    #overlaps = np.array(calculate_overlaps(trajectory, sequence.groundtruth(), (sequence.size) if bounded else None))
    prebbox, confidence = trajectory.prebox_conf(sequence.name)
    gt = sequence.gt 

    # firstframe in each sequence
    overlaps = np.concatenate(([1], np.array([estimateIOU(prebbox[i], gt[i+1] ) for i in range(len(prebbox))])))
    overlaps[np.isnan(overlaps)]=0
    confidence = np.concatenate(([1], np.array(confidence)))


    #n_visible = len([region for region in sequence.groundtruth() if region.type is not RegionType.SPECIAL])
    # sequence.invisible (full-occlusion tag) if invisible= 1 full-occlusion invisible
    visible = np.array(sequence.invisible) < 1
    visible = visible + 0
    try:
        assert len(overlaps) == len(visible) == len(confidence)
    except:
        print("assert not equal")
    first_invisible = np.argwhere(visible == 0)
    all_prre.add_LT(first_invisible, len(overlaps))    
    all_prre.add_list_iou(overlaps)
    all_prre.add_visible(visible)
    all_prre.add_confidence(confidence) 


seq_list = os.listdir('/data1/yjy/rgbd_benchmark/alldata')
seq_list.remove('list.txt')
# all_trackers = [Tracking(tracker) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/results/')]
all_trackers = [Tracking(tracker) for tracker in ['DAL']]
# all_trackers = [Tracking(tracker) for tracker in ['CADMS']]
all_sequence = [Sequence_t(seq) for seq in seq_list]

for trackers in all_trackers:
    print(trackers.name)
    for sequence in all_sequence:
        
        if sequence.name in trackers._seqlist:
            compute_LT_curves(trackers, sequence, trackers._prre)
            #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
        else:
            trackers.lack(sequence.name)
            continue
    
    result = trackers._prre.fscore_LT
    before = result[0]
    after = result[1]
    print('Trackers: {}  {}  fscore: {}'.format(trackers.name, 'before invisible', before))
    print('Trackers: {}  {}  fscore: {}'.format(trackers.name, 'after invisible', after))
    logging.info('Trackers: {}  {}  fscore: {}'.format(trackers.name, 'before invisible', before))
    logging.info('Trackers: {}  {}  fscore: {}'.format(trackers.name, 'after invisible', after))


        

## Attribute evaluation

In [ ]:
def compute_ATT_curves(trajectory: Tracking, sequence: Sequence_t, all_prre: PrRe):

    #overlaps = np.array(calculate_overlaps(trajectory, sequence.groundtruth(), (sequence.size) if bounded else None))
    prebbox, confidence = trajectory.prebox_conf(sequence.name)
    gt = sequence.gt 

    # firstframe in each sequence
    overlaps = np.concatenate(([1], np.array([estimateIOU(prebbox[i], gt[i+1] ) for i in range(len(prebbox))])))
    overlaps[np.isnan(overlaps)]=0
    confidence = np.concatenate(([1], np.array(confidence)))


    #n_visible = len([region for region in sequence.groundtruth() if region.type is not RegionType.SPECIAL])
    # sequence.invisible (full-occlusion tag) if invisible= 1 full-occlusion invisible
    visible = np.array(sequence.invisible) < 1
    visible = visible + 0
    try:
        assert len(overlaps) == len(visible) == len(confidence)
    except:
        print("assert not equal")
    all_attribute = attribute_index(sequence._path, len(overlaps))
    all_prre.add_attribute(all_attribute)    
    all_prre.add_list_iou(overlaps)
    all_prre.add_visible(visible)
    all_prre.add_confidence(confidence) 

def attribute_index(sequencepath, frame_num):
    
    '''
        return: dict {tag} [value]
    '''
    detattribute_list = ['aspect-change.tag', 'background-clutter.tag','depth-change.tag','fast-motion.tag', 'dark-scene.tag', 'moving-view.tag',
                    'deformable.tag', 'out-of-plane.tag', 'out-of-frame.tag', 'partial-occlusion.tag', 'reflective-target.tag', 'size-change.tag', 'similar-objects.tag', 'unassigned.tag']
    cdtbattribute_list = ['fast-motion.tag', 'size-change.tag', 'aspect-change.tag', 'partial-occlusion.tag', 'similar-object.tag', 'out-of-plane.tag', 'depth-change.tag', 'reflective-target.tag',
                    'deformable.tag', 'dark-scene.tag', 'unassigned.tag', 'out-of-frame.tag']
    allattribute_list = list(set(detattribute_list + cdtbattribute_list))
    value_list = []
    att_name = []
    for i, attribute in enumerate(allattribute_list):
        # if attribute == 'background-clutter.tag':
        #     print('ok')
        attpath = os.path.join(sequencepath, attribute)
        try:
            with open(attpath, 'r') as f:
                value = np.loadtxt(f)
            assert len(value) == frame_num
        except:
            value = np.zeros(frame_num)
        value_list.append(value)
        att_name.append(allattribute_list[i])
    dict_attribute = dict(zip(att_name, value_list))
    return dict_attribute

seq_list = os.listdir('/data1/yjy/rgbd_benchmark/alldata')
seq_list.remove('list.txt')
# all_trackers = [Tracking(tracker) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/results/')]
all_trackers = [Tracking(tracker) for tracker in ['DeT']]
all_sequence = [Sequence_t(seq) for seq in seq_list]

output_tracker = []
output_fscore = [[] for i in range(15)]
tag_list = []
for trackers in all_trackers:
    print(trackers.name)
    for sequence in all_sequence:
        
        if sequence.name in trackers._seqlist:
            compute_ATT_curves(trackers, sequence, trackers._prre)
            #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
        else:
            trackers.lack(sequence.name)
            continue
        
    result = trackers._prre.fscore_AT
    print('Trackers: {}'.format(trackers.name))
    for index, tag in enumerate(result):
        print('{}   fscore: {}'.format(tag, result.get(tag)))
        output_fscore[index].append(result.get(tag))
        logging.info('Trackers: {} attribute:{}   fscore: {}'.format(trackers.name, tag, result.get(tag)))
        tag_list.append(tag)
    output_tracker.append(trackers.name)
print('Trackers: ', output_tracker)
outputfile = open('/home/lz/TMM2022/experiment_metric/result/attribute.txt', 'w')
outputfile.writelines('Trackers: {}'.format(output_tracker))
outputfile.writelines('\n')
for i in range(len(output_fscore)):
    str_output = '{} fscore: {}'.format(tag_list[i],output_fscore[i])
    outputfile.writelines(str_output)
    outputfile.writelines('\n')
outputfile.close()


## Overall performance
 

In [14]:
def compute_tpr_curves(trajectory: Tracking, sequence: Sequence_t, all_prre: PrRe):
    
    #overlaps = np.array(calculate_overlaps(trajectory, sequence.groundtruth(), (sequence.size) if bounded else None))
    prebbox, confidence = trajectory.prebox_conf(sequence.name)
    gt = sequence.gt 

    # firstframe in each sequence
    overlaps = np.concatenate(([1], np.array([estimateIOU(prebbox[i], gt[i+1] ) for i in range(len(prebbox))])))
    overlaps[np.isnan(overlaps)]=0
    confidence = np.concatenate(([1], np.array(confidence)))


    #n_visible = len([region for region in sequence.groundtruth() if region.type is not RegionType.SPECIAL])
    # sequence.invisible (full-occlusion tag) if invisible= 1 full-occlusion invisible
    visible = np.array(sequence.invisible) < 1
    visible = visible + 0
    try:
        assert len(overlaps) == len(visible) == len(confidence)
    except:
        print("assert not equal")    
    all_prre.add_list_iou(overlaps)
    all_prre.add_visible(visible)
    all_prre.add_confidence(confidence) 


seq_list = os.listdir('/data1/yjy/rgbd_benchmark/alldata')
seq_list.remove('list.txt')
# all_trackers = [Tracking(tracker) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/results/')]
# all_trackers = [Tracking(tracker) for tracker in ['DAL']]
all_trackers = [Tracking(tracker) for tracker in ['DSKCF']]
all_sequence = [Sequence_t(seq) for seq in seq_list]

for trackers in all_trackers:
    print(trackers.name)
    for sequence in all_sequence:
        
        if sequence.name in trackers._seqlist:
            compute_tpr_curves(trackers, sequence, trackers._prre)
            #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
        else:
            trackers.lack(sequence.name)
            continue
    
    pr,re,fscore = trackers._prre.fscore
    print('Trackers: {}  Seq_num: {} frame_num: {}  pr: {}  re: {}  fscore: {}'.format(trackers.name, trackers._numseq, trackers._prre.count, pr, re, fscore))
    logging.info('Trackers: {}  Seq_num: {} frame_num: {}  pr: {}  re: {}  fscore: {}'.format(trackers.name, trackers._numseq, trackers._prre.count, pr, re, fscore))


        
    
    
    



    
    

DSKCF
Trackers: DSKCF  Seq_num: 164 frame_num: 180373  pr: 0.007326815629597605  re: 0.007666815851331459  fscore: 0.007492960765406419


KeyboardInterrupt: 

In [121]:
#print('name: {}, frame: {}'.format(all_sequence[1].name, all_sequence[1]._numframe))
# prebbox, confidence = all_trackers[2].prebox_conf(all_sequence[2].name)
# for trackers in all_trackers:

#     print('name: {} \t \t number of seq {} '.format(trackers.name, trackers._numseq))     

name: container_room_noocc_1, frame: 1644


## Depth quality evaluation

In [ ]:
# import scipy
# import scipy.io
# mat_det = scipy.io.loadmat('/data1/yjy/rgbd_benchmark/all_benchmark/depthquality/det.mat')
# mat_cdtb = scipy.io.loadmat('/data1/yjy/rgbd_benchmark/all_benchmark/depthquality/cdtb_25.mat')
# mat_stc = scipy.io.loadmat('/data1/yjy/rgbd_benchmark/all_benchmark/depthquality/stc_all.mat') 
# det = mat_det['S'][0]
# cdtb = mat_cdtb['A'][0]
# stc = mat_stc['S'][0]
# sequence_listQ = []
# depthQ = []
# for matfile in [det]:
#     for result in matfile:
#         sequence_name = str(result[0]).split("'")[1]
#         sequence_depthQ =  result[1][0]
#         sequence_listQ.append(sequence_name)
#         depthQ.append(sequence_depthQ)

# for i in range(len(cdtb)-1):
#     sequence_name = str(cdtb[i][0]).split("'")[1]
#     sequence_depthQ = cdtb[i][1][0]
#     sequence_listQ.append(sequence_name)
#     depthQ.append(sequence_depthQ)

# print('ok')

In [ ]:
def compute_DQ(trajectory: Tracking, sequence: Sequence_t, all_prre: PrRe, depthQ):

    #overlaps = np.array(calculate_overlaps(trajectory, sequence.groundtruth(), (sequence.size) if bounded else None))
    prebbox, confidence = trajectory.prebox_conf(sequence.name)
    gt = sequence.gt 

    # firstframe in each sequence
    overlaps = np.concatenate(([1], np.array([estimateIOU(prebbox[i], gt[i+1] ) for i in range(len(prebbox))])))
    overlaps[np.isnan(overlaps)]=0
    confidence = np.concatenate(([1], np.array(confidence)))


    #n_visible = len([region for region in sequence.groundtruth() if region.type is not RegionType.SPECIAL])
    # sequence.invisible (full-occlusion tag) if invisible= 1 full-occlusion invisible
    visible = np.array(sequence.invisible) < 1
    visible = visible + 0
    try:
        assert len(overlaps) == len(visible) == len(confidence) == len(depthQ)
    except:
        print("assert not equal")    
    all_prre.add_list_iou(overlaps)
    all_prre.add_visible(visible)
    all_prre.add_confidence(confidence)
    all_prre.add_depthquality(depthQ) 



'''
    depth quality 
'''
mat_det = scipy.io.loadmat('/data1/yjy/rgbd_benchmark/all_benchmark/depthquality/det.mat')
mat_cdtb = scipy.io.loadmat('/data1/yjy/rgbd_benchmark/all_benchmark/depthquality/cdtb_25.mat')
mat_stc = scipy.io.loadmat('/data1/yjy/rgbd_benchmark/all_benchmark/depthquality/stc_all.mat') 
det = mat_det['S'][0]
cdtb = mat_cdtb['A'][0]
stc = mat_stc['S'][0]
sequence_listQ = []
depthQ = []
for matfile in [det,stc]:
    for result in matfile:
        sequence_name = str(result[0]).split("'")[1]
        sequence_depthQ =  result[1][0]
        sequence_listQ.append(sequence_name)
        depthQ.append(sequence_depthQ)

for i in range(len(cdtb)-1):
    sequence_name = str(cdtb[i][0]).split("'")[1]
    sequence_depthQ = cdtb[i][1][0]
    sequence_listQ.append(sequence_name)
    depthQ.append(sequence_depthQ)
'''
    depth quality
'''
seq_list = os.listdir('/data1/yjy/rgbd_benchmark/alldata')
seq_list.remove('list.txt')
all_trackers = [Tracking(tracker) for tracker in os.listdir('/data1/yjy/rgbd_benchmark/all_benchmark/results/')]
# all_trackers = [Tracking(tracker) for tracker in ['DAL']]
all_sequence = [Sequence_t(sequence_listQ[i]) for i in range(len(sequence_listQ))]

for trackers in all_trackers:
    print(trackers.name)
    for i in range(len(all_sequence)):
        
        if all_sequence[i].name in trackers._seqlist:
            compute_DQ(trackers, all_sequence[i], trackers._prre, depthQ[i])
            #print('{}: length of iou {} '.format(trackers.name, trackers._prre.count))
        else:
            trackers.lack(all_sequence[i].name)
            continue
    
    result = trackers.prre.fscore_DQ
    list_quality = ['high quality', 'medium quality', 'low quality']
    for qualityID in range(len(result)):

        pr,re,fscore = result[qualityID]
        print('Trackers: {} quality level: {}   pr: {}  re: {}  fscore: {}'.format(trackers.name,list_quality[qualityID],pr, re, fscore))
        logging.info('Trackers: {}  quality level: {}  pr: {}  re: {}  fscore: {}'.format(trackers.name, list_quality[qualityID], pr, re, fscore))


        
    

In [35]:
# 0.4 0.8
all_value = 0
num = 0
depth_quality = []
for depth in depthQ:
    for value in depth:
        all_value += value
        num+=1
        depth_quality.append(value)
Z = np.array(depth_quality)
mean = all_value / num

## calculate auc of iou 

In [30]:
import os
# tracker_name = ['DDiMP', 'ATCAIS', 'DRefine', 'SLMD', 'Siam_LTD', 'TSDM', 'iiau_rgbd']
tracker_name = ['ATCAIS19']
gt_dir = '/data1/yjy/rgbd_benchmark/stc_benchmark/RGBDdataset/'
su = Success()
for tracker in tracker_name:
    su.reset()
    tracking_result_dir = '/home/yangjinyu/rgbd_tracker/benchmark_workspace/stc_workspace/results/{}/rgbd-unsupervised'.format(tracker)
    seqlist = os.listdir(tracking_result_dir)
    for seq in seqlist:
        gt_seq = os.path.join(gt_dir, seq, 'groundtruth.txt' )
        pre_seq = os.path.join(tracking_result_dir, '{}/{}_001.txt'.format(seq, seq))
        with open(gt_seq, 'r') as f:
            gt_value = np.loadtxt(f, delimiter=',', skiprows=1)
        with open(pre_seq, 'r') as f:
            pre_value = np.loadtxt(f, delimiter=',', skiprows=1)
        if not len(gt_value) == len(pre_value):
            raise TypeError
        for i in range(len(gt_value)):
            iou = estimateIOU(gt_value[i], pre_value[i])
            su.add_overlap(iou)
    print('{} iou auc: {}'.format(tracker, su.average))
    


/tmp/ipykernel_35807/2209188759.py:76: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  np.sum(i >= thres


ATCAIS19 iou auc: 66.43529551954242


### stc tracker

In [32]:
import os
# tracker_name = ['DDiMP', 'ATCAIS', 'DRefine', 'SLMD', 'Siam_LTD', 'TSDM', 'iiau_rgbd']
tracker_name = ['stc']
gt_dir = '/data1/yjy/rgbd_benchmark/stc_benchmark/RGBDdataset/'
su = Success()
for tracker in tracker_name:
    su.reset()
    tracking_result_dir = '/home/yangjinyu/rgbd_tracker/benchmark_workspace/stc_workspace/results/{}/rgbd-unsupervised'.format(tracker)
    seqlist = os.listdir(tracking_result_dir)
    for seqtxt in seqlist:
        gt_seq = os.path.join(gt_dir, seqtxt.split('.')[0], 'groundtruth.txt' )
        pre_seq = os.path.join(tracking_result_dir, seqtxt)
        with open(gt_seq, 'r') as f:
            gt_value = np.loadtxt(f, delimiter=',', skiprows=1)
        with open(pre_seq, 'r') as f:
            pre_value = np.loadtxt(f, delimiter=',', skiprows=1)
        if not len(gt_value) == len(pre_value):
            raise TypeError
        for i in range(len(gt_value)):
            iou = estimateIOU(gt_value[i], pre_value[i])
            su.add_overlap(iou)
    print('{} iou auc: {}'.format(tracker, su.average))

stc iou auc: 39.79200786112021


### copy result to all_benchmark

In [7]:
seq_list = os.listdir('/data1/yjy/rgbd_benchmark/alldata')
seq_list.remove('list.txt')
# all_trackers = [Tracking(tracker) for tracker in ['DAL']]
all_sequence = [Sequence_t(seq) for seq in seq_list]
count_frame = 0
count_seq = 0
for seq in all_sequence:
    flag, framenum = seq.num_inivisible
    if flag:
        count_seq += 1
        count_frame += framenum
    else:
        continue

print(count_seq, count_frame)

66 98558
